<a href="https://colab.research.google.com/github/phil-mira/NLP/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install peft
!pip install trl
!pip install transformers
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation
!pip install accelerate
!pip3 install autoawq

In [2]:
import json
import os
import torch
from datasets import load_dataset, Dataset, DatasetDict
from peft import LoraConfig, TaskType, get_peft_model, PeftModel, prepare_model_for_kbit_training
from trl import DPOTrainer, DPOConfig
from awq import AutoAWQForCausalLM

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
)

import bitsandbytes
from transformers import BitsAndBytesConfig


ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: cannot open shared object file: No such file or directory
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: cannot op

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [11]:

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Using device: cpu


In [25]:
def chatml_format(example):
    # Format system
    if len(example['system']) > 0:
        message = {"role": "system", "content": example['system']}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""

    # Format instruction
    message = {"role": "user", "content": example['prompt']}

    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)

    # Format chosen answer
    chosen = example['chosen'] + "<|im_end|>\n"

    # Format rejected answer
    rejected = example['rejected'] + "<|im_end|>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }


In [22]:
# Load dataset
def load_preference_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    dataset_dict = {
        "system":[],
        "prompt": [],
        "chosen": [],
        "rejected": []
    }

    for item in data:
        dataset_dict["system"].append("You are a helpful AI assistant working in a school setting that provides suitable answers to a child's questions given their age.")
        dataset_dict["prompt"].append(item["prompt"][0]["content"])
        dataset_dict["chosen"].append(item["chosen"][0]["content"])
        dataset_dict["rejected"].append(item["rejected"][0]["content"])

    return dataset_dict

In [ ]:
def benchmark_model(model_path, tokenizer):

    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        return_dict=True,
        torch_dtype=torch.float16,
    )

    # Load the fine-tuned model
    model = PeftModel.from_pretrained(
        AutoModelForCausalLM.from_pretrained("gpt2", device_map="auto"),
        model_path
    )

    # Load benchmark datasets
    benchmarks = {
        "ai2_arc": load_dataset("ai2_arc", "ARC-Challenge", split="test"),
        "gsm8k": load_dataset("gsm8k", "main", split="test"),
        "hellaswag": load_dataset("hellaswag", "default", split="validation"),
        "truthfulqa": load_dataset("truthful_qa", "multiple_choice", split="validation")
    }

    results = {}

    # Simple evaluation function
    def evaluate_on_dataset(dataset, task_type):
        correct = 0
        total = 0

        for sample in dataset:
            if task_type == "multiple_choice":
                # Handle multiple-choice questions like ARC
                question = sample["question"]
                choices = sample["choices"]["text"]

                best_score = -float('inf')
                best_idx = 0

                for i, choice in enumerate(choices):
                    prompt = f"Question: {question}\nAnswer: {choice}"
                    inputs = tokenizer(prompt, return_tensors="pt").to(device)

                    with torch.no_grad():
                        outputs = model(**inputs)
                        score = outputs.logits.mean().item()

                    if score > best_score:
                        best_score = score
                        best_idx = i

                if best_idx == sample["choices"]["label"]:
                    correct += 1
                total += 1

            elif task_type == "math":
                # Handle math problems like GSM8K
                question = sample["question"]
                reference = sample["answer"]

                prompt = f"Solve this math problem step-by-step:\n{question}\n"
                inputs = tokenizer(prompt, return_tensors="pt").to(device)

                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_length=200,
                        num_return_sequences=1,
                    )

                prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

                # Naive answer extraction - in reality would need more sophisticated parsing
                if str(sample["answer"].split("####")[-1].strip()) in prediction:
                    correct += 1
                total += 1

            elif task_type == "hellaswag":
                # Handle HellaSwag dataset
                context = sample["ctx"]
                endings = sample["endings"]

                best_score = -float('inf')
                best_idx = 0

                for i, ending in enumerate(endings):
                    prompt = f"{context} {ending}"
                    inputs = tokenizer(prompt, return_tensors="pt").to(device)

                    with torch.no_grad():
                        outputs = model(**inputs)
                        score = outputs.logits.mean().item()

                    if score > best_score:
                        best_score = score
                        best_idx = i

                if best_idx == sample["label"]:
                    correct += 1
                total += 1

            elif task_type == "truthfulqa":
                # Handle TruthfulQA dataset
                question = sample["question"]
                choices = sample["mc1_targets"]["choices"]

                best_score = -float('inf')
                best_idx = 0

                for i, choice in enumerate(choices):
                    prompt = f"Question: {question}\nAnswer: {choice}"
                    inputs = tokenizer(prompt, return_tensors="pt").to(device)

                    with torch.no_grad():
                        outputs = model(**inputs)
                        score = outputs.logits.mean().item()

                    if score > best_score:
                        best_score = score
                        best_idx = i

                if best_idx == sample["mc1_targets"]["labels"].index(1):
                    correct += 1
                total += 1

        return correct / total if total > 0 else 0

    # Evaluate on ARC
    results["ai2_arc"] = evaluate_on_dataset(benchmarks["ai2_arc"], "multiple_choice")

    # Evaluate on GSM8K
    results["gsm8k"] = evaluate_on_dataset(benchmarks["gsm8k"], "math")

    # Evaluate on HellaSwag
    results["hellaswag"] = evaluate_on_dataset(benchmarks["hellaswag"], "hellaswag")

    # Evaluate on TruthfulQA
    results["truthfulqa"] = evaluate_on_dataset(benchmarks["truthfulqa"], "truthfulqa")

    # Print results
    print("===== Benchmark Results =====")
    for benchmark, score in results.items():
        print(f"{benchmark}: {score:.4f}")

    # Save results to a JSON file
    with open("benchmark_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4)
    return results

In [ ]:
def compare_models(trained_model_path, tokenizer, questions_file):
    # Load the trained model
    trained_model = PeftModel.from_pretrained(
        AutoModelForCausalLM.from_pretrained("gpt2", device_map="auto"),
        trained_model_path
    )

    # Load the untrained GPT2 model
    untrained_model = AutoModelForCausalLM.from_pretrained("gpt2", device_map="auto")

    # Load the questions
    with open(questions_file, 'r', encoding='utf-8') as f:
        questions = json.load(f)

    results = {"trained_model": {}, "untrained_model": {}}

    # Generate answers for each question
    for idx, question in enumerate(questions):
        prompt = f"Question: {question}\nAnswer:"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        # Generate answer with trained model
        with torch.no_grad():
            trained_output = trained_model.generate(
                **inputs,
                max_length=200,
                num_return_sequences=1,
            )
        trained_answer = tokenizer.decode(trained_output[0], skip_special_tokens=True)
        results["trained_model"][f"question_{idx+1}"] = trained_answer

        # Generate answer with untrained model
        with torch.no_grad():
            untrained_output = untrained_model.generate(
                **inputs,
                max_length=200,
                num_return_sequences=1,
            )
        untrained_answer = tokenizer.decode(untrained_output[0], skip_special_tokens=True)
        results["untrained_model"][f"question_{idx+1}"] = untrained_answer

    # Save results to a JSON file
    with open("model_comparison_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4)

    print("Comparison results saved to model_comparison_results.json")


In [ ]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"

new_model = "./final_dpo_Mistral"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=False)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Load dataset
dataset = load_preference_dataset("dataset.json")

train_size = int(0.9 * len(dataset["prompt"]))

train_dataset = Dataset.from_dict({
    "system": dataset["system"][:train_size],
    "prompt": dataset["prompt"][:train_size],
    "chosen": dataset["chosen"][:train_size],
    "rejected": dataset["rejected"][:train_size]
})

eval_dataset = Dataset.from_dict({
    "system": dataset["system"][train_size:],
    "prompt": dataset["prompt"][train_size:],
    "chosen": dataset["chosen"][train_size:],
    "rejected": dataset["rejected"][train_size:]
})


# Format dataset
train_dataset = train_dataset.map(
    chatml_format,
)

eval_dataset = eval_dataset.map(
    chatml_format,
)


# LoRA Configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
  )

#Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.config.use_cache = False

# Wrap the model with LoRA
model = get_peft_model(model, peft_config, adapter_name="train")

model.add_adaper(peft_config, adapter_name="reference")

training_args = DPOConfig(
    model_adapter_name="train",
    ref_adapter_name="reference",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)


# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

# Train the model
dpo_trainer.train()

# Save the final model
dpo_trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)




In [ ]:
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()


# Benchmark the model
benchmark_model(new_model, tokenizer)

compare_models(new_model, tokenizer, "./test_questions.json")
